In [1]:
import tensorflow as tf
import numpy as np
import cv2

print(tf.__version__)
print(np.__version__)



2.15.1
1.26.4


In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np

# -----------------------------
# 1. Load MNIST dataset
# -----------------------------
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

# -----------------------------
# 2. Preprocess data
# -----------------------------
# Normalize to [0, 1]
x_train = x_train.astype("float32") / 255.0
x_test  = x_test.astype("float32") / 255.0

# Add channel dimension → (samples, 28, 28, 1)
x_train = np.expand_dims(x_train, axis=-1)
x_test  = np.expand_dims(x_test, axis=-1)

# -----------------------------
# 3. Build CNN model
# -----------------------------
model = models.Sequential([
    layers.Input(shape=(28, 28, 1)),

    layers.Conv2D(32, (3, 3), activation="relu"),
    layers.MaxPooling2D((2, 2)),

    layers.Conv2D(64, (3, 3), activation="relu"),
    layers.MaxPooling2D((2, 2)),

    layers.Flatten(),
    layers.Dense(128, activation="relu"),
    layers.Dense(10, activation="softmax")
])

# -----------------------------
# 4. Compile model
# -----------------------------
model.compile(
    optimizer="adam",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"]
)

# -----------------------------
# 5. Train model
# -----------------------------
model.fit(
    x_train,
    y_train,
    epochs=5,
    batch_size=64,
    validation_split=0.1
)

# -----------------------------
# 6. Evaluate model
# -----------------------------
test_loss, test_accuracy = model.evaluate(x_test, y_test)
print("Test accuracy:", test_accuracy)

# -----------------------------
# 7. Save model
# -----------------------------
model.save("mnist_cnn_model.h5")


Epoch 1/5
844/844 [==============================] - 7s 9ms/step - loss: 0.1631 - accuracy: 0.9503 - val_loss: 0.0648 - val_accuracy: 0.9807
Epoch 2/5
844/844 [==============================] - 7s 8ms/step - loss: 0.0507 - accuracy: 0.9844 - val_loss: 0.0332 - val_accuracy: 0.9908
Epoch 3/5
844/844 [==============================] - 8s 9ms/step - loss: 0.0341 - accuracy: 0.9892 - val_loss: 0.0442 - val_accuracy: 0.9873
Epoch 4/5
844/844 [==============================] - 7s 9ms/step - loss: 0.0260 - accuracy: 0.9919 - val_loss: 0.0355 - val_accuracy: 0.9897
Epoch 5/5
313/313 [==============================] - 0s 2ms/step - loss: 0.0253 - accuracy: 0.9926
Test accuracy: 0.9926000237464905


/opt/anaconda3/envs/tf_webcam/lib/python3.10/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [2]:
import tensorflow as tf
import numpy as np
import cv2

# -----------------------------
# 1. Load trained model
# -----------------------------
model = tf.keras.models.load_model("mnist_cnn_model.h5")

# -----------------------------
# 2. Start webcam
# -----------------------------
cap = cv2.VideoCapture(0)

print("Press 'q' to quit")

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Convert to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Crop center region (ROI)
    h, w = gray.shape
    roi = gray[h//2-100:h//2+100, w//2-100:w//2+100]

    # Preprocess ROI
    roi = cv2.resize(roi, (28, 28))
    roi = 255 - roi
    roi = roi / 255.0
    roi = roi.reshape(1, 28, 28, 1)

    # Predict
    prediction = model.predict(roi, verbose=0)
    digit = np.argmax(prediction)

    # Display
    cv2.rectangle(frame,
                  (w//2-100, h//2-100),
                  (w//2+100, h//2+100),
                  (0, 255, 0), 2)

    cv2.putText(frame,
                f"Digit: {digit}",
                (30, 50),
                cv2.FONT_HERSHEY_SIMPLEX,
                1.5,
                (0, 0, 255),
                3)

    cv2.imshow("MNIST Digit Recognition", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# -----------------------------
# 3. Cleanup
# -----------------------------
cap.release()
cv2.destroyAllWindows()


Press 'q' to quit
